In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from roboflow import Roboflow
api_key = input("Digite sua chave de API do Roboflow: ")
rf = Roboflow(api_key=api_key)
project = rf.workspace("gomes-project").project("projeto-libras")
version = project.version(21)
dataset = version.download("yolov8")

In [2]:
paths = []

In [ ]:
paths = [os.path.join("/content/projeto-libras-21/train/images", f) for f in os.listdir("/content/projeto-libras-21/train/images")]

In [3]:
labels = []

In [4]:
for path in paths:
  path_txt = path.replace('/images/', '/labels/').replace('.jpg', '.txt')
  with open(path_txt, 'r') as file:
    linha = file.readline()
  label = linha.split(' ')[0]
  #print(label)
  labels.append([path_txt, label])

In [ ]:
imagem = cv2.imread(paths[99])
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
plt.imshow(imagem)

In [6]:
import csv

In [7]:
num_colunas = 21
colunas = ['class']
for c in range(1, num_colunas+1):
    colunas += ['x{}'.format(c), 'y{}'.format(c), 'z{}'.format(c)]
colunas

['class',
 'x1',
 'y1',
 'z1',
 'x2',
 'y2',
 'z2',
 'x3',
 'y3',
 'z3',
 'x4',
 'y4',
 'z4',
 'x5',
 'y5',
 'z5',
 'x6',
 'y6',
 'z6',
 'x7',
 'y7',
 'z7',
 'x8',
 'y8',
 'z8',
 'x9',
 'y9',
 'z9',
 'x10',
 'y10',
 'z10',
 'x11',
 'y11',
 'z11',
 'x12',
 'y12',
 'z12',
 'x13',
 'y13',
 'z13',
 'x14',
 'y14',
 'z14',
 'x15',
 'y15',
 'z15',
 'x16',
 'y16',
 'z16',
 'x17',
 'y17',
 'z17',
 'x18',
 'y18',
 'z18',
 'x19',
 'y19',
 'z19',
 'x20',
 'y20',
 'z20',
 'x21',
 'y21',
 'z21']

In [ ]:
with open('handgestures.csv', 'w', newline='') as file:
    csv_writer = csv.writer(file, delimiter=",")
    csv_writer.writerow(colunas)

In [8]:
import mediapipe as mp

In [ ]:
mp_hand = mp.solutions.hands
Hand = mp_hand.Hands()
mp_draw = mp.solutions.drawing_utils
connections_style = mp_draw.DrawingSpec(color=(0, 255, 0))

In [ ]:
for c in range(len(paths)):
    imagem = cv2.imread(paths[c])
    imageRGB = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
    label = labels[c][1]
    results = Hand.process(imageRGB)
    if results.multi_hand_landmarks:
        for handlm in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(imageRGB, handlm, mp_hand.HAND_CONNECTIONS, connection_drawing_spec=connections_style)
            hand = handlm.landmark
            hand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand]).flatten())
            hand_row.insert(0, label)
            # Salvar os dados em CSV
            try:
                with open('handgestures.csv', 'a', newline='') as file:
                    csv_writer = csv.writer(file, delimiter=",")
                    csv_writer.writerow(hand_row)
            except Exception as e:
                print(f"Erro ao salvar os landmarks: {e}")